In [ ]:
from typing import TypedDict, Annotated, List

from langchain_ollama import ChatOllama, OllamaEmbeddings
from langchain_community.document_loaders import PyPDFLoader
from langchain_text_splitters import RecursiveCharacterTextSplitter
from langchain_community.vectorstores import FAISS

from langchain_core.tools import tool
from langchain_core.messages import HumanMessage, BaseMessage

from langgraph.graph import StateGraph, START, END
from langgraph.graph.message import add_messages
from langgraph.prebuilt import ToolNode, tools_condition


# =========================
# 1. LOAD PDF
# =========================

PDF_PATH = "sample.pdf"  # <-- ensure this exists

loader = PyPDFLoader(PDF_PATH)
documents = loader.load()


# =========================
# 2. SPLIT DOCUMENT
# =========================

text_splitter = RecursiveCharacterTextSplitter(
    chunk_size=800,
    chunk_overlap=150
)

docs = text_splitter.split_documents(documents)


# =========================
# 3. EMBEDDINGS & VECTOR DB
# =========================

embeddings = OllamaEmbeddings(
    model="nomic-embed-text"  # ✅ available in your ollama list
)

vectorstore = FAISS.from_documents(docs, embeddings)
retriever = vectorstore.as_retriever(search_kwargs={"k": 4})


# =========================
# 4. TOOL
# =========================

@tool
def search_docs(query: str) -> str:
    """Search the PDF and return relevant context."""
    results = retriever.get_relevant_documents(query)
    return "\n\n".join(doc.page_content for doc in results)


tools = [search_docs]


# =========================
# 5. LLM
# =========================

llm = ChatOllama(
    model="qwen2.5:3b",  # or mistral:7b
    temperature=0.2
)

llm_with_tools = llm.bind_tools(tools)


# =========================
# 6. GRAPH STATE
# =========================

class AgentState(TypedDict):
    messages: Annotated[List[BaseMessage], add_messages]


# =========================
# 7. NODES
# =========================

def chatbot(state: AgentState):
    response = llm_with_tools.invoke(state["messages"])
    return {"messages": [response]}


tool_node = ToolNode(tools)


# =========================
# 8. GRAPH
# =========================

graph = StateGraph(AgentState)

graph.add_node("chatbot", chatbot)
graph.add_node("tools", tool_node)

graph.add_edge(START, "chatbot")
graph.add_conditional_edges("chatbot", tools_condition)
graph.add_edge("tools", "chatbot")
graph.add_edge("chatbot", END)

app = graph.compile()


# =========================
# 9. CHAT LOOP
# =========================

if __name__ == "__main__":
    print("📄 RAG Chatbot ready (type 'exit' to quit)\n")

    state = {"messages": []}

    while True:
        user_input = input("You: ")
        if user_input.lower() in {"exit", "quit"}:
            break

        state["messages"].append(HumanMessage(content=user_input))
        state = app.invoke(state)

        print("Bot:", state["messages"][-1].content)
